In [ ]:
%matplotlib inline
import signac
import flow
import numpy as np
from matplotlib import pyplot as plt
import copy

from project import Project
print(signac.__version__)
print(flow.__version__)

mplColors = ['tab:blue', 'tab:orange', 'tab:green', 'tab:red', 'tab:purple',
             'tab:brown', 'tab:pink', 'tab:gray', 'tab:olive', 'tab:cyan']

In [ ]:
project = Project()
print(project)

benchmark_doc = signac.core.jsondict.JSONDict(filename='benchmark.json')

In [ ]:
benchmark = benchmark_doc()
time = benchmark['time']

generations = list(sorted(map(int, time)))
t = np.array([time[str(g)] for g in generations])
t -= t[0]  # normalize by start time
dt = t[1:] - t[:-1]
dg = generations[1:]
fig = plt.figure(figsize=(6,6))
ax1 = fig.add_subplot(111)
ax1.plot(generations, t, ls='--')
ax2 = ax1.twinx()
ax2.plot(generations[1:], dt)

ax1.set(xlabel="generation", ylabel="total time")
ax1.xaxis.label.set_size(16)
ax1.yaxis.label.set_size(16)
ax2.set(ylabel="time/generation")
ax2.yaxis.label.set_size(16)
fig.tight_layout()
plt.show()

save data for later plotting

In [ ]:
d = copy.copy(project.doc)
g = np.array([int(key) for key in d.time])
t = np.array([d.time[key] for key in d.time])
nj = np.array([d.njobs[key] for key in d.njobs])
# print("mean: {}; std: {}".format(np.mean(nj), np.std(nj)))
dt = t[1:] - t[:-1]
dg = g[1:]
dn = nj[1:]
# save data
np.save("Bash.Serial.time.npy", t)
np.save("Bash.Serial.generation.npy", g)
np.save("Bash.Serial.njobs.npy", nj)
fig = plt.figure(figsize=(8,6))
ax1 = fig.add_subplot(111)
ax1.plot(g, t, color='tab:blue', label='Total Time')
ax2 = ax1.twinx()
# ax2.plot(dg, dt)
# ax2.plot(dg, dn)
ax2.plot(dg, np.divide(dt, dn), color='tab:orange', label='time/generation/njobs')

ax1.set(xlabel="generation", ylabel="time (total)")
ax1.xaxis.label.set_size(16)
ax1.yaxis.label.set_size(16)
ax2.set(ylabel="time")
ax2.yaxis.label.set_size(16)
ax1.legend(loc=2)
ax2.legend(loc=4)
plt.show()

In [ ]:
# load and compare

## Python

### parallel
ppt = np.load("Python.Parallel.time.npy")
ppg = np.load("Python.Parallel.generation.npy")
ppn = np.load("Python.Parallel.njobs.npy")

ppt[:] = ppt[:] - ppt[0]
ppdt = ppt[1:] - ppt[:-1]
ppdg = ppg[1:]
ppdn = ppn[1:]
ppdt = np.divide(ppdt, ppdn)

### serial
pst = np.load("Python.Serial.time.npy")
psg = np.load("Python.Serial.generation.npy")
psn = np.load("Python.Serial.njobs.npy")

pst[:] = pst[:] - pst[0]
psdt = pst[1:] - pst[:-1]
psdg = psg[1:]
psdn = psn[1:]
psdt = np.divide(psdt, psdn)

## Bash

### parallel
bpt = np.load("Bash.Parallel.time.npy")
bpg = np.load("Bash.Parallel.generation.npy")
bpn = np.load("Bash.Parallel.njobs.npy")

bpt[:] = bpt[:] - bpt[0]
bpdt = bpt[1:] - bpt[:-1]
bpdg = bpg[1:]
bpdn = bpn[1:]
bpdt = np.divide(bpdt, bpdn)

### serial
bst = np.load("Bash.Serial.time.npy")
bsg = np.load("Bash.Serial.generation.npy")
bsn = np.load("Bash.Serial.njobs.npy")

bst[:] = bst[:] - bst[0]
bsdt = bst[1:] - bst[:-1]
bsdg = bsg[1:]
bsdn = bsn[1:]
bsdt = np.divide(bsdt, bsdn)

# plot

fig = plt.figure(figsize=(8,6))
ax1 = fig.add_subplot(111)
ax1.plot(ppg, ppt, color=mplColors[0], label='Python/Parallel', ls="-.", linewidth=2)
ax1.plot(psg, pst, color=mplColors[1], label='Python/Serial' ,ls="-.", linewidth=2)
ax1.plot(bpg, bpt, color=mplColors[2], label='Bash/Parallel', ls="-.", linewidth=2)
ax1.plot(bsg, bst, color=mplColors[3], label='Bash/Serial', ls="-.", linewidth=2)

ax2 = ax1.twinx()
ax2.plot(ppdg, ppdt, color=mplColors[0])
ax2.plot(psdg, psdt, color=mplColors[1])
ax2.plot(bpdg, bpdt, color=mplColors[2])
ax2.plot(bsdg, bsdt, color=mplColors[3])

# ax2.plot(dg, dt)
# ax2.plot(dg, dn)
# ax2.plot(dg, np.divide(dt, dn), color='tab:orange', label='time/generation/njobs')

ax1.set(xlabel="generation", ylabel="time (total)")
ax1.set_xlim((0,500))
ax1.xaxis.label.set_size(16)
ax1.yaxis.label.set_size(16)

ax2.set(ylabel="time/generation/njobs")
ax2.yaxis.label.set_size(16)

ax1.legend(loc=2)
# ax2.legend(loc=4)
plt.show()

In [ ]:
print(project.doc)
# project.print_status(detailed=True)

In [ ]:

def isEligible(job):
    if 'eligible' in job.document:
        if job.document.eligible == True:
            return True
        else:
            return False
    else:
        return False


def isNotEligible(job):
    if 'eligible' in job.document:
        if job.document.eligible == True:
            return False
        else:
            return True
    else:
        return False

In [ ]:
with project.open_job(id="a53fd35b5d71b10dd3e27c7c35e426c3") as job:
    print(job.document)